## Setup input talbe

In [30]:
%%writefile setup_table.py
from kinetica_proc import ProcData
import test_environment as te
import shutil

"""setup the table example_loan_data for UDF"""

te.ingest_loan_data()

Overwriting setup_table.py


In [ ]:
%run python setup_table.py

## Register and run the training UDF

In [29]:
%%writefile Register_execute_H2Otrain.py
import gpudb
import collections
import sys

INPUT_TABLE = "example_loan_data"
host_ip="127.0.0.1" 

#register UDF
def main(h_db):
    proc_name = 'H2o_Train'
    
    filelist=["H2O_train.py","kinetica_proc.py"] # put the main python script in the first place
    # Read proc code in as bytes and add to a file data array
    files = {}
    for ifile in filelist:
        with open(ifile,'rb') as f:
            files[ifile]=f.read()
    # Connect to Kinetica
    h_db = gpudb.GPUdb(encoding = 'BINARY', host = host_ip, port ="9191")
    # Remove proc if it exists from a prior registration
    if h_db.has_proc(proc_name)['proc_exists']:
        h_db.delete_proc(proc_name)
    print("Registering proc...")
    response = h_db.create_proc(proc_name, 'distributed', files, 'python', [filelist[0]], {})

    ####################
    #Execute UDF
    print("Executing proc...")
    response = h_db.execute_proc(proc_name, {}, {}, [INPUT_TABLE], {}, [], {})
    print(response)

if __name__=="__main__":
    h_db = gpudb.GPUdb(encoding = 'BINARY', host = host_ip, port = 9191)
    main(h_db)

Overwriting Register_execute_H2Otrain.py


In [27]:
%run python Register_execute_H2Otrain.py

Registering proc...
Executing proc...
{'run_id': '18', 'status_info': {'status': 'OK', 'message': '', 'data_type': 'execute_proc_response', 'response_time': 0.00115}}
